<a href="https://colab.research.google.com/github/lengochai97/thesis/blob/master/notebooks/feature_construction/42_News_Click_Counts_Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connect to Google Drive

In [0]:
%%capture

import google.colab.drive

google.colab.drive.mount('/content/gdrive', force_remount=True)

# Install Spark and dependencies

In [0]:
import os

os.environ['HADOOP_VERSION'] = '2.7'
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME'] = '/opt/spark'
os.environ['SPARK_VERSION'] = '2.4.3'

In [0]:
%%capture

!wget -qN https://archive.apache.org/dist/spark/spark-$SPARK_VERSION/spark-$SPARK_VERSION-bin-hadoop$HADOOP_VERSION.tgz
!tar -xzf spark-$SPARK_VERSION-bin-hadoop$HADOOP_VERSION.tgz -C /opt
!rm spark-$SPARK_VERSION-bin-hadoop$HADOOP_VERSION.tgz
!rm -rf /opt/spark
!ln -s /opt/spark-$SPARK_VERSION-bin-hadoop$HADOOP_VERSION /opt/spark
!pip install -q findspark

# Create SparkSession

In [0]:
import findspark

findspark.init()

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local[*]').getOrCreate()

# Read files

In [0]:
import json

import pyspark.sql.functions as F
import pyspark.sql.types as T

In [0]:
DATA_PATH = '/content/gdrive/My Drive/dataset/adressa/one_week'

In [0]:
with open(os.path.join(DATA_PATH, 'schema', 'news_click_counts.json')) as file:
  news_click_counts_schema = T.StructType.fromJson(json.load(file))

In [0]:
df_news_click_counts = spark.read.json(os.path.join(DATA_PATH, 'news_click_counts'), schema=news_click_counts_schema)

# Check results

In [0]:
df_news_click_counts.show(truncate=False)

+----------------------------------------+----------+----------------+----------------+----------------+----------------+
|newsId                                  |time      |newsClickCount1H|newsClickCount6H|newsClickCount1D|newsClickCount1W|
+----------------------------------------+----------+----------------+----------------+----------------+----------------+
|ccd8230aececb03e231403dbba86069bbe45d1bf|1483247867|0               |0               |0               |0               |
|ccd8230aececb03e231403dbba86069bbe45d1bf|1483247936|1               |1               |1               |1               |
|ccd8230aececb03e231403dbba86069bbe45d1bf|1483248011|1               |1               |1               |1               |
|ccd8230aececb03e231403dbba86069bbe45d1bf|1483248047|1               |1               |1               |1               |
|ccd8230aececb03e231403dbba86069bbe45d1bf|1483248088|1               |1               |1               |1               |
|ccd8230aececb03e231403d

## Number of (`newsId`, `time`) pairs

In [0]:
df_news_click_counts.count()

13630336

In [0]:
df_news_click_counts.select(F.column('newsId'), F.column('time')).distinct().count()

13630336

## Number of items

In [0]:
df_news_click_counts.select(F.column('newsId')).distinct().count()

1034

## Disk usage

In [0]:
!du -sh /content/gdrive/My\ Drive/dataset/adressa/one_week/news_click_counts

2.1G	/content/gdrive/My Drive/dataset/adressa/one_week/news_click_counts
